In [18]:
import os
import glob
import re

from bs4 import BeautifulSoup
import pandas as pd

In [10]:
def string2geocoord(string):
    coord = re.sub('S|N|E|W', '.', string)    
    if 'S' in string or 'W' in string:
        return '-' + coord
    else:
        return coord    

    
def is_company_data(table):
    return tb.find('label').get('id') == 'labelNúmero/Nome'


def is_erb_data(table):
    return tb.find('label').text == 'Estação'

In [11]:
DATA_DIR = '../data'
DATA_FILES = glob.glob(os.path.join(DATA_DIR, '*.html'))
print(f'Encontrados {len(DATA_FILES)} arquivos de dados.')

Encontrados 27 arquivos de dados.


In [14]:
dados_br = pd.DataFrame()
converters = {'Estação': str}
for n, datafile in enumerate(DATA_FILES):
    print(n, '-', datafile)
    data = {}
    soup = BeautifulSoup(open(datafile, encoding='Latin1').read(), 'lxml')
    tables = soup.find_all('table')
    for tb in tables:
        if is_company_data(tb):
            company_id = tb.find('label').text        
            company_number = company_id[:7].strip()
            company_name = company_id[10:].strip()
            company_sgl = company_name.split(' ')[0]        
        else:
            if is_erb_data(tb):
                HTML = str(tb)
                df = pd.read_html(HTML, header=0, converters=converters)[0]
                df.insert(0, 'nom_operadora', company_name)
                df.insert(0, 'sgl_operadora', company_sgl)
                df.insert(0, 'num_operadora', company_number)            
                data[company_name] = df
    dados_br = pd.concat([dados_br, pd.concat([data[i] for i in data], ignore_index=True)], ignore_index=True)

0 - ../data/BA.html
1 - ../data/SP.html
2 - ../data/RS.html
3 - ../data/TO.html
4 - ../data/RO.html
5 - ../data/AM.html
6 - ../data/AP.html
7 - ../data/RR.html
8 - ../data/PR.html
9 - ../data/SC.html
10 - ../data/AL.html
11 - ../data/RN.html
12 - ../data/AC.html
13 - ../data/SE.html
14 - ../data/PB.html
15 - ../data/MS.html
16 - ../data/MG.html
17 - ../data/PA.html
18 - ../data/DF.html
19 - ../data/RJ.html
20 - ../data/CE.html
21 - ../data/MA.html
22 - ../data/PE.html
23 - ../data/GO.html
24 - ../data/ES.html
25 - ../data/PI.html
26 - ../data/MT.html


In [23]:
dados_br.head()

,num_operadora,sgl_operadora,nom_operadora,Estação,Nome,UF,Município,Bairro,Logradouro,Latitude,Longitude,Data Cadastro,Data 1º Lic.,Última Licença
0,1278410,OI,OI MÓVEL S.A. - EM RECUPERAÇÃO JUDICIAL,641073011,BABRR0026 / 3GBABRR0862 / 4GBABRR5862,BA,Salvador,Cabula,Rua Leila Diniz - nº 150,-12.571720,-38.274240,16/11/2001,09/08/2002,30/05/2016
1,1278410,OI,OI MÓVEL S.A. - EM RECUPERAÇÃO JUDICIAL,641076797,BBAROM0101 / 3GBAROM1354 / 4GBAROM5354,BA,Salvador,BOMFIM,RUA TRAVASSO DO MEIO - nº 41,-12.552750,-38.301970,19/11/2001,09/08/2002,30/05/2016
2,1278410,OI,OI MÓVEL S.A. - EM RECUPERAÇÃO JUDICIAL,641088949,BALFS0410 / 3GBALFS1038 / 4GBALFS5038,BA,Lauro de Freitas,BURAQUINHO,RUA JULIO M. DOS SANTOS - nº S/Nº,-12.524680,-38.171260,21/11/2001,09/08/2002,09/01/2017
3,1278410,OI,OI MÓVEL S.A. - EM RECUPERAÇÃO JUDICIAL,641089023,BARMS0032 / 3GBARMS0032 / 4GBARMS5032,BA,Salvador,COUTOS,AVENIDA AFRANIO PEIXOTO - nº S/N°,-12.511900,-38.283200,21/11/2001,09/08/2002,30/05/2016
4,1278410,OI,OI MÓVEL S.A. - EM RECUPERAÇÃO JUDICIAL,641089066,BAFSA0302 / 4GBAFSA5302,BA,Feira de Santana,RIBEIRO,TRAVESSA DOS CONTABILISTAS - nº 100,-12.152549,-38.574860,21/11/2001,09/08/2002,30/05/2016


In [19]:
dados_br.Latitude = dados_br.Latitude.map(string2geocoord)
dados_br.Longitude = dados_br.Longitude.map(string2geocoord)

In [24]:
dados_br.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 86243 entries, 0 to 86242
Data columns (total 14 columns):
num_operadora     86243 non-null object
sgl_operadora     86243 non-null object
nom_operadora     86243 non-null object
Estação           86243 non-null object
Nome              86154 non-null object
UF                86243 non-null object
Município         86243 non-null object
Bairro            84656 non-null object
Logradouro        86243 non-null object
Latitude          86243 non-null object
Longitude         86243 non-null object
Data Cadastro     86243 non-null object
Data 1º Lic.      84241 non-null object
Última Licença    80080 non-null object
dtypes: object(14)
memory usage: 9.2+ MB


In [27]:
dados_br.groupby(['sgl_operadora', 'UF']).Estação.count()

sgl_operadora  UF
ALGAR          GO      23
               MG     579
               MS       7
               SP      86
CLARO          AC      55
               AL     193
               AM     192
               AP      33
               BA    1153
               CE     486
               DF     409
               ES     287
               GO     717
               MA     350
               MG    1621
               MS     257
               MT     319
               PA     367
               PB     208
               PE     635
               PI     263
               PR     992
               RJ    1706
               RN     220
               RO     185
               RR      28
               RS    1173
               SC     632
               SE     110
               SP    3818
                     ... 
TELEFÔNICA     SP    4856
               TO     102
TIM            AC      46
               AL     299
               AM     236
               AP      50
               BA   

In [25]:
dados_br.to_csv('../data/dataset-erb-BR.csv', sep=';', encoding='Latin1')